In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal, norm
from scipy import optimize

## Определяем целевую функцию

Наша цель - оптимизация функции правдоподобия, взвешенную многомерным нормальным распределением. Для начала подготовим данные в удобную для вычисления функции форму.

In [2]:
def par_matrix(par_df, quest_pair, thresholds, pattern):
    '''
    Преобразует датафрейм с параметрами вопросов в матрицы
    
    Input:
    par_df - pd.DataFrame с параметрами вопросов (параметры: trait - шкала, к которой относится вопрос; lmbd - факторная нагрузка вопроса 
    на шкалу,se - стандартная ошибка вопроса). Строки - вопросы, столбцы - параметры
    quest_pair - np.array с указанием того, к какой паре и на каком месте относится вопрос. Вопрос справа всегда идет первым в формуле 4
    (положительный в формуле), вопрос слева идет вторым в формуле 4 (отрицательным). Строки - вопросы, столбцы - пары
    thresholds - np.array с 5 границами для ответов
    pattern - np.array размерности число шкал с ответами кандидата на пары (баллы 1, 2, 3, 4, 5)
    
    Output:
    tuple, содержащий:
    lmbd_pair_mat - np.array размерности число шкал х число пар с факторными нагрузками, соответствующими парам пунктов
    threshold_pairs - np.array размерности число кандидатов х число пар с значениями границ с учетом паттерна ответов
    se_pairs - np.array размерности число пар с корнем сумм квадратов стандартных ошибок для вопросов в паре
    '''
    
    #Записываем размерности данных (число шкал, пар и вопросов)
    n_traits = len(par_df["trait"].unique().tolist())
    n_pairs = quest_pair.shape[1]
    n_quest = par_df.shape[0]
    
    #Распределяем факторные нагрузки по матрице размерности число шкал х число вопросов
    lmbd_quest_mat = np.zeros([n_traits, n_quest])
    lmbd_quest_mat[par_df["trait"] - 1, par_df.index] = par_df["lmbd"]
    
    #Сворачиваем факторные нагрузки в матрицу размерности число шкал х число пар с учетом того, какой вопрос предъявляется справа, а какой слева
    lmbd_pair_mat = np.dot(lmbd_quest_mat, quest_pair)
    
    #Распределяем значения границ по ответам кандидата
    threshold_pairs = thresholds[pattern - 1]
    
    #
    se_pairs = np.sqrt(np.dot(par_df["se"] ** 2, abs(quest_pair)))
    
    return lmbd_pair_mat, threshold_pairs, se_pairs

Исходя из этого пропишем вычисление функции правдоподобия

In [3]:
def loglik_fun(person_par, lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize = True):
    '''
    Считает логарифмическую функцию правдоподобия по результатам кандидата по шкалам опросника и другим фиксированным параметрам
    
    Input:
    person_par - аргумент функции, np.array размерности число шкал с предварительным расчетом результата по шкалам
    lmbd_pair_mat - np.array размерности число шкал х число пар с факторными нагрузками, соответствующими парам пунктов
    threshold_pairs - np.array размерности число кандидатов х число пар с значениями границ с учетом паттерна ответов
    se_pairs - np.array размерности число пар с корнем сумм квадратов стандартных ошибок для вопросов в паре
    sigma - np.array размерности число шкал х число шкал - матрица корреляций шкал между собой
    minimize - bool, минимизируем мы функцию или максимизируем
    
    Output:
    log_lik_post - float, значение функции правдоподобия в точке person_par
    '''
    #Необходимые нам размерности
    n_traits = person_par.shape[0]
    
    #Вычисляем значение логарифма плотности многомерного нормального распределения в точке person_par
    multinorm_pdf = multivariate_normal.logpdf(person_par, mean = np.zeros(n_traits), cov = sigma)
    
    #Вычисляем входное значение для вычисления вероятности паттерна ответов
    person_lmbd_pairs = np.dot(person_par, lmbd_pair_mat)
    numerator = -threshold_pairs + person_lmbd_pairs
    probs_input = numerator / se_pairs
    
    #Вычисляем логарифм вероятностей каждого ответа, использую кумулятивную функцию нормального распределения
    log_probs = norm.logcdf(probs_input)
    
    #Суммируем эти величины
    log_lik = np.sum(log_probs)
    
    #Добавляем взвешивающий компонент
    log_lik_post = log_lik + multinorm_pdf
    
    if minimize:
        return -log_lik_post
    
    return log_lik_post

Прописываем градиент функции

In [4]:
def loglik_grad(person_par, lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize = True):
    '''
    Рассчитывает градиент логарифмической функции правдоподобия по результатам кандидата по шкалам опросника и другим фиксированным параметрам
    
    Input:
    person_par - аргумент функции, np.array размерности число шкал с предварительным расчетом результата по шкалам
    lmbd_pair_mat - np.array размерности число шкал х число пар с факторными нагрузками, соответствующими парам пунктов
    threshold_pairs - np.array размерности число кандидатов х число пар с значениями границ с учетом паттерна ответов
    se_pairs - np.array размерности число пар с корнем сумм квадратов стандартных ошибок для вопросов в паре
    sigma - np.array размерности число шкал х число шкал - матрица корреляций шкал между собой
    minimize - bool, минимизируем мы функцию или максимизируем
    
    Output:
    loglik_grad - np.array размерности , каждый элемент которого - это частная производная функции loglik_fun по элементам person_par
    '''
    
    #Вычисляем производную функции многомерного распределения
    multinorm_grad = - np.dot(np.linalg.inv(sigma), person_par.T)
    
    #Вычисляем фактор угла наклона производной для всех пар
    trait_pair_grad = lmbd_pair_mat / se_pairs
    
    #Вычисляем входное значение для вычисления вероятности паттерна ответов
    person_lmbd_pairs = np.dot(person_par, lmbd_pair_mat)
    numerator = -threshold_pairs + person_lmbd_pairs
    probs_input = numerator / se_pairs
    
    #Вычисляем компоненты градиента, вычисленные по плотности и функции вероятности
    density_mult = (norm.pdf(probs_input) / norm.cdf(probs_input))
    
    #Суммируем все компоненты градиента
    loglik_grad = multinorm_grad.T + np.sum(trait_pair_grad * density_mult, axis = 1)
    
    if minimize:
        return -loglik_grad
    return loglik_grad
    

Подгружаем данные - параметры вопросов:

In [5]:
par_df = pd.read_csv("data/quest_pars2.csv", sep = ";")
print(par_df.info())
par_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
trait    60 non-null int64
lmbd     60 non-null float64
se       60 non-null float64
dtypes: float64(2), int64(1)
memory usage: 1.5 KB
None


,trait,lmbd,se
0,1,1.0,0.8
1,1,1.1,0.7
2,1,0.9,0.6
3,1,1.2,0.9
4,1,1.1,1.0


Подгружаем матрицу распределения вопросов по парам:

In [6]:
quest_pair = pd.read_csv("data/quest_pairs3.csv", sep = ";", header = None)
print(quest_pair.info())
print(quest_pair.head())
quest_pair = quest_pair.as_matrix()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 30 columns):
0     60 non-null int64
1     60 non-null int64
2     60 non-null int64
3     60 non-null int64
4     60 non-null int64
5     60 non-null int64
6     60 non-null int64
7     60 non-null int64
8     60 non-null int64
9     60 non-null int64
10    60 non-null int64
11    60 non-null int64
12    60 non-null int64
13    60 non-null int64
14    60 non-null int64
15    60 non-null int64
16    60 non-null int64
17    60 non-null int64
18    60 non-null int64
19    60 non-null int64
20    60 non-null int64
21    60 non-null int64
22    60 non-null int64
23    60 non-null int64
24    60 non-null int64
25    60 non-null int64
26    60 non-null int64
27    60 non-null int64
28    60 non-null int64
29    60 non-null int64
dtypes: int64(30)
memory usage: 14.1 KB
None
   0   1   2   3   4   5   6   7   8   9  ...  20  21  22  23  24  25  26  27  \
0   0   0   0   0   1   0   0   0   0   0 ...   0  

C:\Users\Nikolay\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


Добавляем значения границ и паттерн ответов, а также оставшиеся параметры на вход функциям

In [9]:
threshold = np.array([-1.5, -0.5, 0, 0.5, 1.5])
#pattern = np.array([1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

patterns = pd.read_csv("data/pattern2.csv", sep = ";", header = None).as_matrix()
pattern0 = patterns[0, :]
#pattern1 = patterns[1, :]

lmbd_pair_mat, threshold_pairs, se_pairs = par_matrix(par_df, quest_pair, threshold, pattern0)

sigma = np.eye(5)
minimize = True

C:\Users\Nikolay\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


Инициализируем результаты по шкалам и проверяем расчет функции и градиента

In [10]:
person_par0 = np.zeros(5)

print(loglik_fun(person_par0, lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize))
print(loglik_grad(person_par0, lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize))

23.14621427872789
[ 1.74007086  1.88928153 -4.03366472 -0.85011454  2.35491526]


Проверяем правильность расчета градиента с помощью численного метода (результат должен быть близок к 0)

In [11]:
optimize.check_grad(loglik_fun, loglik_grad, person_par0, lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize)

2.3572067739696156e-07

In [12]:
person_par1 = np.array([1, 0, -1, -2, 2])

optimize.check_grad(loglik_fun, loglik_grad, person_par1, lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize)

5.946502787994131e-07

In [13]:
person_par2 = np.array([2, 2, 2, 2, 2])

optimize.check_grad(loglik_fun, loglik_grad, person_par2, lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize)

7.500315919398657e-07

Посмотрим на результаты оптимизации функции - несколько вариантов для проверки

In [14]:
optimize.fmin_bfgs(loglik_fun, x0 = person_par0, args = (lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize), fprime = loglik_grad)

Optimization terminated successfully.
         Current function value: 21.634995
         Iterations: 10
         Function evaluations: 12
         Gradient evaluations: 12


array([-0.25515146, -0.23006901,  0.34332119, -0.06078786, -0.36094884])

In [15]:
optimize.fmin_l_bfgs_b(loglik_fun, x0 = person_par0, args = (lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize), fprime = loglik_grad)

(array([-0.25514832, -0.23007063,  0.34332018, -0.06078586, -0.36094811]),
 21.634995046271797,
 {'grad': array([ 2.49824798e-05, -2.12934305e-05, -1.19050971e-05,  1.55958236e-05,
         -1.25487557e-06]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 12,
  'nit': 10,
  'warnflag': 0})

In [16]:
optimize.minimize(loglik_fun, x0 = person_par0, args = (lmbd_pair_mat, threshold_pairs, se_pairs, sigma, minimize), jac = loglik_grad, method = "L-BFGS-B")

      fun: 21.634995046271797
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.49824798e-05, -2.12934305e-05, -1.19050971e-05,  1.55958236e-05,
       -1.25487557e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 12
      nit: 10
   status: 0
  success: True
        x: array([-0.25514832, -0.23007063,  0.34332018, -0.06078586, -0.36094811])

In [17]:
import IPython

print(IPython.sys_info())

{'commit_hash': '2e1cca5bc',
 'commit_source': 'installation',
 'default_encoding': '1251',
 'ipython_path': 'C:\\Users\\Nikolay\\Anaconda3\\lib\\site-packages\\IPython',
 'ipython_version': '7.3.0',
 'os_name': 'nt',
 'platform': 'Windows-10-10.0.17763-SP0',
 'sys_executable': 'C:\\Users\\Nikolay\\Anaconda3\\python.exe',
 'sys_platform': 'win32',
 'sys_version': '3.7.2 (default, Feb 21 2019, 17:35:59) [MSC v.1915 64 bit '
                '(AMD64)]'}
